In [ ]:
# import os
# # Ensure the directory exists before changing to it
# directory = 'Robust Network Intrusion Detection through Explainable Artificial Intelligence (XAI)'
# if not os.path.exists(directory):
#     os.makedirs(directory)  # Create the directory if it doesn't exist
# os.chdir(directory)  # Change to the directory where files are stored (main + utility)
save_loc = './results/'    			# location to save results
data_loc = 'dataset'              # location of dataset

import numpy as np
import matplotlib.pyplot as plt
import pickle
import xgboost
import tensorflow as tf
import shap
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler, MinMaxScaler

import utility_funcs as utf

np.random.seed(10)
# tf.random.set_seed(10)


data_kdd = utf.read_KDD(data_loc)
with open(f"{save_loc}/model.pkl", "rb") as f:
    model = pickle.load(f)
# from sklearn.neural_network import MLPClassifier

# # instantiate an MLP with default settings (you can tweak hidden_layer_sizes, activation, solver, etc.)
# model = MLPClassifier(random_state=10, max_iter=200)

# # fit on the KDD train set
# model.fit(data_kdd['X_train'], data_kdd['Y_train_bin'])




# # # train model using default settings of XGBoost
# # model = xgboost.XGBClassifier(use_label_encoder=False, objective="binary:logistic", seed= 10)
# # model.fit(data_kdd['X_train'], data_kdd['Y_train_bin'])

# # compute performance statistics of the model, store in 'results_model' object
results_model = {}
results_model['y_pred_train'] = model.predict(data_kdd['X_train'])
# # results_model['y_pred_test'] = model.predict(data_kdd['X_test'])
# # results_model['y_pred_test_21'] = model.predict(data_kdd['X_test_21'])

# # results_model['performance_model_train'] = utf.compute_performance_stats(data_kdd['Y_train_bin'], results_model['y_pred_train'])
# # results_model['performance_model_test'] = utf.compute_performance_stats(data_kdd['Y_test_bin'], results_model['y_pred_test'])
# # results_model['performance_model_test_21'] = utf.compute_performance_stats(data_kdd['Y_test_bin_21'], results_model['y_pred_test_21'])

# # SAVE DATA
# pickle.dump(data_kdd, open("{}/data_nsl.pkl".format(save_loc), "wb"))
# pickle.dump(model, open("{}/model.pkl".format(save_loc), "wb"))


# load the trained model


# calculate how many new attacks the XGBoost model can identify, NB during deployment, we have no way of knowing these are new attacks
z = np.zeros(len(data_kdd['Y_test_bin']),)
z[data_kdd['new_attack_locs']] = 1
temp = np.multiply(z, results_model['y_pred_test'])
results_model['num_new_attacks_detected'] =   np.sum(temp)                                      # correctly identified 1297 new attacks (TP)
results_model['TPR_new_attacks'] = np.round(np.sum(temp)/len(data_kdd['new_attack_locs']) , 2)  # i.e., correctly identified 35% of new attacks (TPR)


""" ************************* Create AutoEncoder based on SHAP data **************************** """
# # compute SHAP values of model across train and test sets
# results_AE_SHAP = {}
# explainer = shap.TreeExplainer(model, data_kdd['X_train'], feature_perturbation = "interventional", model_output='probability') # NB output='probability' decomposes inputs among Pr(Y=1='Attack'|X)
# results_AE_SHAP['shap_train'] = explainer.shap_values(data_kdd['X_train'])
# results_AE_SHAP['shap_test'] = explainer.shap_values(data_kdd['X_test'])

# Compute example explanation for a Probe attack - NB SHAP package requires us to wrap everything into a single object before plotting
idx = data_kdd['Y_train'][data_kdd['Y_train']=='ipsweep'].index # get all locations where ipsweep attack occur (i.e., main attack class = Probe)

class Object(object):
    pass

exp = Object
exp.feature_names = data_kdd['feature_names']
exp.base_values = explainer.expected_value
exp.data = data_kdd['X_train'].loc[idx[0]]
exp.values = results_AE_SHAP['shap_train'][idx[0]]

FileNotFoundError: [Errno 2] No such file or directory: './Robust Network Intrusion Detection through Explainable Artificial Intelligence (XAI)/dataset/KDDTrain+.txt'